<a href="https://colab.research.google.com/github/alanbseo/MFAworkshop/blob/main/MFA3_2_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

과제를 위해 공통으로 실행해야 하는 부분을 첫 번쩨 셀에 정리

In [ ]:
# import packages
import os
import matplotlib.pyplot as plt
import numpy as np


# import pandas and geopandas package
import pandas as pd

try:
  import geopandas as gpd
except: # 설치되어 있지 않을 경우 이리로 분기하여 패키지를 설치
  !pip install geopandas
  import geopandas as gpd

# import rasterio pacakge
try:
  import rasterio
except:
  !pip install rasterio
  import rasterio

# import required functions
from rasterio import transform
from rasterio import features
from rasterio.enums import Resampling

from google.colab import drive

# Mount Gdrive
# 이번에는 이미 마운트 되어 있는지 확인하고
if not (os.path.exists('/content/gdrive/')):
  print("Mount Gdrive")
  drive.mount('/content/gdrive/') # 마운트 되어있지 않을 때만 다시 시도


# set working directory
default_path = '/content/gdrive/MyDrive/Net zero built environment/MFA workshop' # 이 경로는 사용자 마다 상이할 수 있음

print("Change working directory to the Google Drive folder")
os.chdir(default_path)
print(os.getcwd()) # print the current working directory



# Gross area data for Jongno-gu (Feb 2023)
path_data_grossarea = "Data/Data_100m/Jongno-gu/(B100)국토통계_건축물-연면적-(격자) 100M_서울특별시 종로구_202303/nlsp_021002006.shp"
grossarea_gdf = gpd.read_file(path_data_grossarea, encoding = "UTF-8")

# Create a base 100 m grid
dummy_shp = grossarea_gdf # 여기서 원본 데이터를 dummy에 할당, crs 정보도 옮겨옴
bbox = dummy_shp.total_bounds # 벡터 데이터의 바운딩 박스를 래스터 만들 때 쓰려고 bbox 변수에 할당
xmin, ymin, xmax, ymax = bbox # bbox의 정보를 4개의 double 변수에 옮김. 파이썬 스타일..
                              # 오른쪽 변수에서 리턴되는 값이 double형 네 개이고 그게 xmin 등 4개의 실수형 변수 값으로 됨

print(bbox.__class__) # Numpy N-d array 클래스이고
print(bbox.shape)     # 실수형 자료 4개를 가지고 있음
print(xmin.__class__) # 받은 xmin등은 Numpy Float64. 64비트 플로트 정수. Float는 Double 보다 메모리를 아껴서 저장하는 실수형 타입..

res = 100 # desired resolution
w = (xmax - xmin) // res
h = (ymax - ymin) // res


# float type for continuous data
out_meta_float64 = {
    "driver": "GTiff",
    "dtype": "float64", # 64bit float
    "height": h,
    "width": w,
    "count": 1,
    "crs": dummy_shp.crs,
    "transform": transform.from_bounds(xmin, ymin, xmax, ymax, w, h),
    "compress": 'lzw'
}



# Read the rasterised Volume and Code data
code_out_fn = './images/code_100m.tif'

with rasterio.open(code_out_fn) as src:
    #print(src.profile)
    code_img = src.read(1)

# read the outcome rasters
grossarea_100m_r = rasterio.open('images/grossarea_100m.tif')
grossarea_100m_img = grossarea_100m_r.read(1)




Change working directory to the Google Drive folder
/content/gdrive/.shortcut-targets-by-id/1gDT-7hKMl37JnTVecJLmI9yB2rUI7rAo/Net zero built environment/MFA workshop
<class 'numpy.ndarray'>
(4,)
<class 'numpy.float64'>


In [ ]:
MI_tb = pd.read_excel('Data/MI by structure_v2.xlsx', sheet_name = 'MI')
MI_tb = MI_tb.fillna(0) # NaN을 0으로 바꿔주자

EC_tb = pd.read_excel('Data/MI by structure_v2.xlsx', sheet_name = 'GWP')
EC_tb = EC_tb.fillna(0) # NaN을 0으로 바꿔주자